In [5]:
# !pip install missingno

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm    
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale , StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix , accuracy_score , mean_squared_error , r2_score , roc_auc_score , roc_curve , classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.model_selection import KFold
import missingno as msno
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
import warnings
warnings.simplefilter(action='ignore')
sns.set()
plt.style.use("ggplot")
%matplotlib inline

In [7]:
# read the database for dir
df = pd.read_csv("diabetes.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# independent feature -> 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age', 
# dependent feature   -> 'Outcome'
# descriptive statistics of the dataset
df.describe()

In [ ]:
# (rows , column)

In [ ]:
df.shape  

In [ ]:
# to get distribution of outcome which mean how mean preson get 0 or 1
# value_counts return unique value

In [ ]:
df['Outcome'].value_counts()*100/len(df)

In [ ]:
df.Outcome.value_counts()*100/len(df)

In [ ]:
df.Outcome.value_counts()

In [ ]:
# len(df) it return total number of rows 
# Converts the raw counts into percentages, by multiplying by 100 and dividing by the total.

In [ ]:
df.Outcome.value_counts()*100/len(df)


In [ ]:
# plot the historgram of the age variable 

In [ ]:
plt.figure(figsize=(8,7))
plt.xlabel('Age',fontsize=10)
plt.ylabel('Count',fontsize=10)
df.Age.hist(edgecolor="black")

In [ ]:
 df.Age.max()

In [ ]:
df.Age.min()

In [ ]:
print("Max Age : " + str( df.Age.max()))
print("Min Age : " + str( df.Age.min()))

In [ ]:
df.columns

In [ ]:
# Density plot 
fig,ax=plt.subplots(4,2 , figsize=(20,20))
sns.distplot(df.Pregnancies,bins=20,ax=ax[0,0],color="red")
sns.distplot(df.Glucose,bins=20,ax=ax[0,1],color="red")
sns.distplot(df.BloodPressure,bins=20,ax=ax[1,0],color="red")
sns.distplot(df.SkinThickness,bins=20,ax=ax[1,1],color="red")
sns.distplot(df.Insulin,bins=20,ax=ax[2,0],color="red")
sns.distplot(df.BMI,bins=20,ax=ax[2,1],color="red")
sns.distplot(df.DiabetesPedigreeFunction,bins=20,ax=ax[3,0],color="red")
sns.distplot(df.Age,bins=20,ax=ax[3,1],color="red")


In [ ]:
df.columns

In [ ]:
df.groupby("Outcome").agg({'Pregnancies':'mean'})

In [ ]:
df.groupby("Outcome").agg({'Pregnancies':'max'})

In [ ]:
df.groupby("Outcome").agg({'Glucose':'mean'})


In [ ]:
df.groupby("Outcome").agg({'Glucose':'max'})

In [ ]:
f,ax =plt.subplots(1,2,figsize=(18,8))
df['Outcome'].value_counts().plot.pie(explode=[0,0.1],autopct="%1.1f%%",ax=ax[0],shadow=True)
ax[0].set_title('target')
ax[0].set_ylabel('')
sns.countplot(x='Outcome', data=df, ax=ax[1])
ax[1].set_title('Outcome')
plt.show()


In [ ]:
df.corr()

In [ ]:
f,ax=plt.subplots(figsize=[20,15])
sns.heatmap(df.corr(), annot=True, fmt='.2f', ax=ax, cmap='magma')
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

In [ ]:
# Data preprocessing part 

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']]=df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']].replace(0, np.NaN)

In [ ]:
df.isnull().sum()

In [ ]:
msno.bar(df, color="red")

In [ ]:
def median_target(var):
    temp=df[df[var].notnull()]
    temp=temp[[var,'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp

In [ ]:

columns = df.columns
columns = columns.drop('Outcome')
for i in columns:
    median_target(i)
    df.loc[(df['Outcome']==0) & (df[i].isnull()), i] = median_target(i)[i][0]
    df.loc[(df['Outcome']==1) & (df[i].isnull()), i] = median_target(i)[i][1]

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# pair plot
p = sns.pairplot(df, hue="Outcome")

In [ ]:
# Outlier Detection 
# IQR
# 50%
# 24.65-->25% + 50%
# 24.65-->25%
for feature in df :
    Q1 =df[feature].quantile(0.25)
    Q3 =df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1-1.5*IQR
    upper = Q3+1.5*IQR
    if df[(df[feature]>upper)].any(axis=None):
        print(feature, "yes")
    else:
        print(feature, "No")

In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(x=df["Insulin"],color="red")

In [ ]:
Q1 = df.Insulin.quantile(0.25)
Q3 = df.Insulin.quantile(0.75)
IQR = Q3 -Q1
lower = Q1-1.5*IQR
upper = Q3+1.5*IQR
df.loc[df['Insulin']>upper,"Insulin"] = upper

In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(x=df["Insulin"],color="red")

In [ ]:
# LOF --> local outlier factor
lof = LocalOutlierFactor(n_neighbors=10)
lof.fit_predict(df)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(x=df["Pregnancies"],color="red")

In [ ]:
df_score= lof.negative_outlier_factor_
np.sort(df_score)[0:20]

In [ ]:
thresold = np.sort(df_score)[7]

In [ ]:
outlier = df_score>thresold

In [ ]:
outlier

In [ ]:
df=df[outlier]

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(x=df["Pregnancies"],color="red")

In [ ]:
# Feature Enginnering

In [ ]:
NewBMI = pd.Series(["Underweight","Normal","Overweight","Obesity 1","Obesity 2","Obesity 3"], dtype = "category")

In [ ]:
NewBMI

In [ ]:
df['NewBMI'] = NewBMI
df.loc[df["BMI"]<18.5,"NewBMI"] = NewBMI[0]
df.loc[(df["BMI"]>18.5) & df["BMI"] <=24.9, "NewBMI"] = NewBMI[1]
df.loc[(df["BMI"]>24.9) & df["BMI"] <=29.9, "NewBMI"] = NewBMI[2]
df.loc[(df["BMI"]>29.9) & df["BMI"] <=34.9, "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"]>34.9) & df["BMI"] <=39.9, "NewBMI"] = NewBMI[4]
df.loc[df["BMI"]>39.9,"NewBMI"] = NewBMI[5]


In [ ]:
df.head()

In [ ]:
# if insulin >= 16 & insulin <= 166 --> normal
def set_insuline(row):
    if row["Insulin"]>=16 and row["Insulin"]<=166:
        return "Noraml"
    else:
        return "Abnormal"

In [ ]:
df = df.assign(NewInsulinScore=df.apply(set_insuline, axis=1))
# df["NewInsulinScore"] = df.apply(set_insuline, axis=1)

In [ ]:
df.head()

In [ ]:
# Some interval were determined according to the glucose variable  and these were assigned catergorical
NewGlucose = pd.Series(["Low","Normal","Overweight","Secert","High"], dtype = "category")
df['NewGlucose'] = NewGlucose
df.loc[df["Glucose"]<=70,"NewGlucose"] = NewGlucose[0]
df.loc[(df["Glucose"]>70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df.loc[(df["Glucose"]>99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df.loc[df["Glucose"]>126,"NewGlucose"] = NewGlucose[3]


In [ ]:
df.head()

In [ ]:
# One hot encoding 

In [ ]:
df = pd.get_dummies(df, columns=["NewBMI", "NewInsulinScore", "NewGlucose"], drop_first=True)


In [ ]:
df.head()

In [ ]:
df[ df.select_dtypes(bool).columns ] = df.select_dtypes(bool).astype(int)


In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
categorical_df = df[['NewBMI_Obesity 1',
       'NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight',
       'NewBMI_Underweight', 'NewInsulinScore_Noraml', 'NewGlucose_Low',
       'NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secert']]

In [ ]:
categorical_df.head()

In [ ]:
y = df['Outcome']
X = df.drop(['Outcome','NewBMI_Obesity 1',
       'NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight',
       'NewBMI_Underweight', 'NewInsulinScore_Noraml', 'NewGlucose_Low',
       'NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secert'], axis=1)

In [ ]:
cols = X.columns
index = X.index

In [ ]:
X.head()

In [ ]:
transformer = RobustScaler().fit(X)
X = transformer.transform(X)
X = pd.DataFrame(X, columns = cols, index = index)

In [ ]:
X.head()

In [ ]:
X = pd.concat([X, categorical_df], axis=1)

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0) 

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Machine Learning Algorithm 

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)